# Classification Model Evaluation

- different domains have different needs

    - iPhone face id -- high precision
    - fraud detection -- high recall

In [1]:
import pandas as pd

df = pd.DataFrame({
    'actual': ['coffee', 'no coffee', 'no coffee', 'coffee', 'coffee', 'coffee', 'no coffee', 'coffee'],
    'prediction': ['no coffee', 'no coffee', 'coffee', 'coffee', 'coffee', 'coffee', 'no coffee', 'no coffee'],
})
df

,actual,prediction
0,coffee,no coffee
1,no coffee,no coffee
2,no coffee,coffee
3,coffee,coffee
4,coffee,coffee
5,coffee,coffee
6,no coffee,no coffee
7,coffee,no coffee


## Confusion Matrix

In [5]:
pd.crosstab(df.prediction, df.actual)

actual,coffee,no coffee
prediction,,
coffee,3,1
no coffee,2,2


- TP: predicted coffee + actual is coffee
- FP: predicted coffee, but they didn't like coffee
- FN: predicted no coffee, but really they liked coffee
- TN: predicted no coffee, actual no coffee

- our choice of positive and negative is arbitrary
- the labels / layout of the confusion matrix varies

## Metrics

- **accuracy**: (TP + TN) / (TP + TN + FP + FN)
    - (3 + 2) / (3 + 1 + 2 +2) = 62.5%
- **precision**: TP / (TP + FP)
    - 3 / (3 + 1) = 75%
    - FP is more costly than FN
- **recall**: TP / (TP + FN)
    - 3 / (3 + 2) = 60%
    - FN is more costly than FP
    
Imagine you're bringing coffee to meeting, which metric would we choose? It depends

Outcomes:

- FP: Buy a coffee for someone who won't drink it
- FN: Don't buy a coffee for someone who wanted one
- TP: Buy a coffee for someone who will drink it
- TN: Don't buy a coffee for someone who wouldn't drink it anyway

Scenarios

- lola: really good coffee, but super expensive
    - cost of a FP is higher than FN
    - precision is better here because buying a cup of coffee for someone who won't drink it is expensive
    - We want to be sure about our positive predictions
- taco cabana: bad coffee, but cheap
    - cost of a FN is higher than FP
    - recall because the coffee is cheap, its not bad to buy a cheap coffee for someone who won't drink it; worse to not get someone coffee who wanted it
- meeting with super important client
    - cost of FN is higher, because they might be offended if we dont' get them coffee
    - cost of FN == not signing a contract
    - recall

What if we just don't buy coffee or buy coffee for everyone? Baseline model

In [7]:
df.actual.value_counts()

coffee       5
no coffee    3
Name: actual, dtype: int64

In [8]:
df['baseline'] = 'coffee'

In [9]:
df

,actual,prediction,baseline
0,coffee,no coffee,coffee
1,no coffee,no coffee,coffee
2,no coffee,coffee,coffee
3,coffee,coffee,coffee
4,coffee,coffee,coffee
5,coffee,coffee,coffee
6,no coffee,no coffee,coffee
7,coffee,no coffee,coffee


### Python Implementation

In [12]:
# model accuracy
(df.actual == df.prediction).mean()

0.625

In [14]:
# baseline accuracy
(df.actual == df.baseline).mean()

0.625

In [17]:
# precision -- how good are our positive predictions
# precision -- model performance | pred +
subset = df[df.prediction == 'coffee']
print(subset)
(subset.prediction == subset.actual).mean()

      actual prediction baseline
2  no coffee     coffee   coffee
3     coffee     coffee   coffee
4     coffee     coffee   coffee
5     coffee     coffee   coffee


0.75

In [19]:
# recall -- how often do we get the actual positive cases
# recall -- model performance | actual +
subset = df[df.actual == 'coffee']
print(subset)
(subset.prediction == subset.actual).mean()

   actual prediction baseline
0  coffee  no coffee   coffee
3  coffee     coffee   coffee
4  coffee     coffee   coffee
5  coffee     coffee   coffee
7  coffee  no coffee   coffee


0.6

What will the precision and recall of our baseline model that always predicts + be?

In [21]:
# precision
subset = df[df.baseline == 'coffee']
print(subset)
(subset.baseline == subset.actual).mean()

      actual prediction baseline
0     coffee  no coffee   coffee
1  no coffee  no coffee   coffee
2  no coffee     coffee   coffee
3     coffee     coffee   coffee
4     coffee     coffee   coffee
5     coffee     coffee   coffee
6  no coffee  no coffee   coffee
7     coffee  no coffee   coffee


0.625

In [23]:
# recall
subset = df[df.actual == 'coffee']
print(subset)
(subset.baseline == subset.actual).mean()

   actual prediction baseline
0  coffee  no coffee   coffee
3  coffee     coffee   coffee
4  coffee     coffee   coffee
5  coffee     coffee   coffee
7  coffee  no coffee   coffee


1.0

## Spam Example

Predict whether an email is spam or not.

- positive case: spam
- negative case: not spam

Outcomes:

- TP: predict that a message is spam, and it really is
- FP: predict that a message is spam, but its really not
- FN: predict that a message is not spam, but it really is
- TN: predict that a message is not spam, and it really is not spam

Which has a higher cost, a false negative or false positive?

FP -- its **_bad_** to put bill in accounting's email in your spam folder

Precision: because we want to be certain a message really is spam when we send it to the spam folder

## Phishing Example

Predict whether an email is a phishing attempt. When we predict positive, show an additional banner

- positive case: phishing attempt
- negative case: legit email

Outcomes:

- FP: Show a warning about phishing on a legit email
- TP: Show a warning about phishing on a phishing email
- FN: Don't show a warning on a phishing email
- TN: Don't show a warning on a legit email

Which has the highest cost? FN: not showing a warning on a phishing email

The cost of showing a banner on a legit email is less than not showing the warning on a phishing.

Optimize for recall in our phishing detection model.

## What does "positive" mean?

In [30]:
positive = 'coffee'

# accuracy -- overall hit rate
model_accuracy = (df.prediction == df.actual).mean()
baseline_accuracy = (df.baseline == df.actual).mean()

# precision -- how good are our positive predictions?
# precision -- model performance | predicted positive
subset = df[df.prediction == positive]
model_precision = (subset.prediction == subset.actual).mean()
subset = df[df.baseline == positive]
baseline_precision = (subset.baseline == subset.actual).mean()

# recall -- how good are we at detecting actual positives?
# recall -- model performance | actual positive
subset = df[df.actual == positive]
model_recall = (subset.prediction == subset.actual).mean()
baseline_recall = (subset.baseline == subset.actual).mean()


print(f'   model accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')
print()
print(f'   model recall: {model_recall:.2%}')
print(f'baseline recall: {baseline_recall:.2%}')
print()
print(f'model precision: {model_precision:.2%}')
print(f'baseline precision: {baseline_precision:.2%}')

   model accuracy: 62.50%
baseline accuracy: 62.50%

   model recall: 60.00%
baseline recall: 100.00%

model precision: 75.00%
baseline precision: 62.50%


## Recap

In short:

- optimize for **precision** when you want to be sure about your positive predictions
- optimize for **recall** when you don't want to miss positive cases 